In [3]:
import cvxpy as cp
import numpy as np

# 1. Definir as matrizes dos vértices do sistema
A1 = np.array([[5.3, 5.0], [-5.3, -12.7]])
A2 = np.array([[-13.0, 20.9], [-0.6, -14.2]])
B1 = np.array([[-0.2], [-1.2]])
B2 = np.array([[0.0], [0.1]])

# Dimensões do sistema (n=estados, m=entradas)
n, m = A1.shape[0], B1.shape[1]

# 2. Escolher um valor para o parâmetro xi (ξ)
# O problema como formulado é uma BMI (Bilinear Matrix Inequality) se xi for uma variável.
# Uma abordagem comum é fixar xi e resolver o problema de viabilidade LMI resultante.
# Um valor pequeno e positivo para xi é geralmente um bom ponto de partida.
xi = 0.1
# Se o problema for inviável, pode ser necessário ajustar este valor.

print(f"Tentando resolver o problema de viabilidade LMI com xi = {xi}...")

# 3. Definir as variáveis de otimização do CVXPY
# W1, W2 devem ser simétricas e definidas positivas
W1 = cp.Variable((n, n), symmetric=True)
W2 = cp.Variable((n, n), symmetric=True)
# X é uma matriz geral n x n
X = cp.Variable((n, n))
# Z é a matriz de ganho modificada m x n (Z = K*X)
Z = cp.Variable((m, n))

# 4. Montar a lista de restrições LMI
constraints = [
    W1 >> 0,
    W2 >> 0,
]

# A LMI deve ser satisfeita para cada vértice (i=1, 2)
for A, B, W in [(A1, B1, W1), (A2, B2, W2)]:
    # Blocos da LMI
    M11 = A @ X + X.T @ A.T + B @ Z + Z.T @ B.T
    M12 = W - X.T + xi * (A @ X + B @ Z)
    M22 = -xi * (X + X.T)
    
    # Montar a LMI de blocos
    LMI = cp.bmat([
        [M11, M12],
        [M12.T, M22]
    ])
    
    constraints.append(LMI << 0)

# 5. Definir e resolver o problema de viabilidade
# O objetivo é encontrar qualquer solução, então minimizamos uma constante.
objective = cp.Minimize(0)
problem = cp.Problem(objective, constraints)

# Usar um solver SDP como SCS ou MOSEK
problem.solve(solver=cp.SCS)

# 6. Apresentar os resultados
if problem.status == cp.OPTIMAL:
    print("\nProblema de viabilidade resolvido com sucesso!")
    print("Um controlador estabilizante foi encontrado.")
    
    # Calcular o ganho do controlador K = Z * X^-1
    try:
        X_val = X.value
        Z_val = Z.value
        K = Z_val @ np.linalg.inv(X_val)
        print("\nMatriz de ganho do controlador K:")
        print(K)
        
        # Opcional: Verificar a estabilidade do sistema em malha fechada nos vértices
        print("\nVerificando autovalores do sistema em malha fechada nos vértices:")
        cl_sys1 = A1 + B1 @ K
        cl_sys2 = A2 + B2 @ K
        eig1 = np.linalg.eigvals(cl_sys1)
        eig2 = np.linalg.eigvals(cl_sys2)
        print(f"Autovalores de (A1 + B1*K): {np.round(eig1, 4)}")
        print(f"Autovalores de (A2 + B2*K): {np.round(eig2, 4)}")
        
        if np.all(np.real(eig1) < 0) and np.all(np.real(eig2) < 0):
            print("\nConclusão: O controlador estabiliza robustamente os vértices do sistema.")
        else:
            print("\nAtenção: O controlador encontrado não estabiliza os vértices.")

    except np.linalg.LinAlgError:
        print("\nNão foi possível calcular o ganho K porque a matriz X é singular.")
        print("Matriz X encontrada:")
        print(X.value)

elif problem.status == cp.INFEASIBLE:
    print(f"\nO problema é inviável para xi = {xi}.")
    print("Tente um valor diferente para xi (maior ou menor).")
    print("Se continuar inviável, pode não existir um controlador que satisfaça estas condições LMI específicas.")
else:
    print("\nO solver não conseguiu resolver o problema. Status:", problem.status)


Tentando resolver o problema de viabilidade LMI com xi = 0.1...

Problema de viabilidade resolvido com sucesso!
Um controlador estabilizante foi encontrado.

Não foi possível calcular o ganho K porque a matriz X é singular.
Matriz X encontrada:
[[0. 0.]
 [0. 0.]]
